# Pre-Processing the Files

#### Import Python packages 

In [11]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity 
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Question-1 : Gat the artist, song title and song's length in music app history that was heard during sessionId=338, itemInSession=4

#### Column Names: We need Name of the artist, title of the song and length of the track on query upon sessionId and itemInSession. Hence we will select artist_name , song_title , song_length, sessionId and itemInSession as the name of the columns.
#### Primary Key: The PRIMARY key for the table should uniquely identify each row in the table. For us we need results based on sessionId and itemInSession; so we neeed these both as the primary key

In [8]:
## create table if not exist for the Query 1
query = "CREATE TABLE IF NOT EXISTS song_library"
query = query + "(sessionid int, iteminsession int, artist text, song text, length double ,PRIMARY KEY (sessionid, iteminsession ))"
try:
    session.execute(query)
except Exception as e:
    print(e)               

### Insert data for the Query 1

In [9]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_library (sessionid, iteminsession, artist, song,length )"
        query = query + "VALUES (%s, %s, %s, %s , %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]) ))

#### Answer 1:

In [12]:
query = "select artist, song, length from song_library WHERE sessionid = 338 and iteminsession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)    

t = PrettyTable(['Artist', 'Song', 'Length'])
for row in rows:
    t.add_row([row.artist, row.song, row.length])
print(t)    

+-----------+---------------------------------+----------+
|   Artist  |               Song              |  Length  |
+-----------+---------------------------------+----------+
| Faithless | Music Matters (Mark Knight Dub) | 495.3073 |
+-----------+---------------------------------+----------+


### Question-2: Get name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Column Names: We need Name of the artist, title of the song and user first name and lastname on query upon userid and sessionid. Hence we will select artist_name , song_title , song_length, sessionId and itemInSession as the name of the columns.
#### Primary Key: The PRIMARY key for the table should uniquely identify each row in the table. For us we need results based on userid  and sessionId; we will use userid as partition key and sessionid as a cluster column, but since we also need result been sorted by itemInSession, we will also use column itemInSession as a cluster column 

In [13]:
## create table if not exist for the Query 2
query = "CREATE TABLE IF NOT EXISTS user_library"
query = query + "(userid int,sessionid int,itemInSession int,firstname text, lastname text, artist text, song text, PRIMARY KEY (userid, sessionid, itemInSession ))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Insert data for the Query 2

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_library (userid,sessionid,itemInSession, firstname,lastname,artist, song )"
        query = query + "VALUES (%s, %s, %s, %s , %s, %s, %s)"
        session.execute(query, (int(line[10]),int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]  ))   

#### Answer 2:

In [15]:
query = "select artist, song, firstname, lastname from user_library WHERE userid = 10 and sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
t = PrettyTable(['Artist', 'Song', 'User_firstame','User_lastname'])
for row in rows:
    t.add_row([row.artist, row.song, row.firstname, row.lastname])
print(t)    

+-------------------+------------------------------------------------------+---------------+---------------+
|       Artist      |                         Song                         | User_firstame | User_lastname |
+-------------------+------------------------------------------------------+---------------+---------------+
|  Down To The Bone |                  Keep On Keepin' On                  |     Sylvie    |      Cruz     |
|    Three Drives   |                     Greece 2000                      |     Sylvie    |      Cruz     |
| Sebastien Tellier |                      Kilometer                       |     Sylvie    |      Cruz     |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |     Sylvie    |      Cruz     |
+-------------------+------------------------------------------------------+---------------+---------------+


### Question-3: Get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Column Names: We need  title of the song and user first name and lastname on query upon song title. Hence we will select song,userid, firstname, lastname as the name of the columns.
#### Primary Key: For us we need results based on song title; The PRIMARY key for the table should uniquely identify each row in the table. because of that we need to use song and userid in Primary Key.

In [16]:
## create table if not exist for the Query 3
query = "CREATE TABLE IF NOT EXISTS music_library"
query = query + "( song text, userid int, firstname text, lastname text, PRIMARY KEY (song, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Insert data for the Query 3

In [17]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO music_library (song, userid, firstname,lastname)"
        query = query + "VALUES (%s, %s, %s, %s )"
        session.execute(query, (line[9] ,int(line[10]), line[1], line[4],))                     

#### Answer 3:

In [18]:
query = "select firstname, lastname from music_library WHERE song ='All Hands Against His Own' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
t = PrettyTable(['User_firstame','User_lastname'])
for row in rows:
    t.add_row([row.firstname, row.lastname])
print(t)     

+---------------+---------------+
| User_firstame | User_lastname |
+---------------+---------------+
|   Jacqueline  |     Lynch     |
|     Tegan     |     Levine    |
|      Sara     |    Johnson    |
+---------------+---------------+


### Drop the tables before closing out the sessions

In [15]:
drop_table_song_library = "drop table song_library"
drop_table_user_library = "drop table user_library"
drop_table_music_library= "drop table music_library"
try:
    rows = session.execute(drop_table_song_library)
    rows = session.execute(drop_table_user_library)
    rows = session.execute(drop_table_music_library)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [16]:
session.shutdown()
cluster.shutdown()